<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EPAT Session 2

**Executive Program in Algorithmic Trading**

**_Vectorized Backtesting_**

Prof. Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

<a href="https://home.tpq.io/certificates/pyalgo" target="_blank"><img src="https://hilpisch.com/pyalgo_cover_color.png" width="300px" align="left" border="1px"></a>

## Basic Imports

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment', None)

## Reading Financial Data

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'  # EOD data
# url = 'http://hilpisch.com/aiif_eikon_id_data.csv'  # intraday data

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.columns

## Regime Detection

In [ ]:
sym = 'EUR='

In [ ]:
data = pd.DataFrame(raw[sym])

In [ ]:
data.plot(figsize=(10, 6));

### Features

In [ ]:
data['returns'] = np.log(data[sym] / data[sym].shift(1))

In [ ]:
window = 20

In [ ]:
data['trend'] = data['returns'].rolling(window).mean()  # rolling time series momentum

In [ ]:
data['risk'] = data['returns'].rolling(window).std()  # rolling volatilitycv

In [ ]:
cols = ['trend', 'risk']

In [ ]:
# data[cols] = (data[cols] - data[cols].mean()) / data[cols].std()  # Gaussian Normalization

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.plot(kind='scatter', x='risk', y='trend', figsize=(10, 6));

### Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(n_clusters=4)  # unsupervised learning

In [ ]:
model.fit(data[cols])  # unsupervised learning

In [ ]:
data['regime'] = model.predict(data[['trend', 'risk']])

In [ ]:
data['regime'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x=data['risk'], y=data['trend'],
           c=data['regime'], cmap='coolwarm')
plt.xlabel('risk')
plt.ylabel('trend');

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x=data.index, y=data[sym],
           c=data['regime'], cmap='coolwarm',
           marker='.');

## Advanced Trading Strategy

### Data Preprocessing

In [ ]:
sym = 'EUR='

In [ ]:
data = pd.DataFrame(raw[sym])

In [ ]:
# data.plot(figsize=(10, 6));

### Features

In [ ]:
data['returns'] = np.log(data[sym] / data[sym].shift(1))

In [ ]:
data['direction'] = np.sign(data['returns'])

In [ ]:
# data['direction']

In [ ]:
window = 20

In [ ]:
data['mom'] = data['returns'].rolling(window).mean()  # rolling time series momentum

In [ ]:
data['vol'] = data['returns'].rolling(window).std()  # rolling volatility

In [ ]:
features = ['returns']#'mom', 'vol']

In [ ]:
lags = 5
cols = []
for f in features:
    for lag in range(1, lags + 1):
        col = f'{f}_lag_{lag}'
        data[col] = data[f].shift(lag)
        cols.append(col)

In [ ]:
cols

In [ ]:
data.dropna(inplace=True)

In [ ]:
# data[cols] = (data[cols] - data[cols].mean()) / data[cols].std()  # Gaussian Normalization

In [ ]:
# data.head()

### Model Fitting (In-Sample)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model = GaussianNB()  # supervised learning

In [ ]:
model = LogisticRegression(C=100)  # supervised learning

In [ ]:
model = MLPClassifier(max_iter=500)  # supervised learning

In [ ]:
%time model.fit(data[cols], data['direction'])  # supervised learning

In [ ]:
data['prediction'] = model.predict(data[cols])

In [ ]:
data['prediction'].value_counts()

In [ ]:
accuracy_score(data['prediction'], data['direction'])

### Vectorized Backtesting (In-Sample)

In [ ]:
data['strategy'] = data['returns'] * data['prediction']

In [ ]:
data[['returns', 'strategy']].sum()  # sum of log returns

In [ ]:
data[['returns', 'strategy']].sum().apply(np.exp)  # gross performance

In [ ]:
data[['returns', 'strategy']].sum().apply(np.exp) - 1  # net performance

In [ ]:
data[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6));  # gross performance over time

### Train-Test Split

In [ ]:
len(data)

In [ ]:
split = int(0.9 * len(data))
split

In [ ]:
train = data.iloc[:split].copy()
test = data.iloc[split:].copy()

In [ ]:
mu, std = train[cols].mean(), train[cols].std()
train[cols] = (train[cols] - mu) / std
test[cols] = (test[cols] - mu) / std

### Training (In-Sample)

In [ ]:
model = MLPClassifier(hidden_layer_sizes=[100], max_iter=500,
                      # random_state=500,
                      shuffle=False)  # supervised learning

In [ ]:
# MLPClassifier?

In [ ]:
model.fit(train[cols], train['direction'])

In [ ]:
train['prediction'] = model.predict(train[cols])

In [ ]:
train['strategy'] = train['prediction'] * train['returns']

In [ ]:
accuracy_score(train['prediction'], train['direction'])

In [ ]:
train[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6));  # gross performance over time

### Testing (Out-of-Sample)

In [ ]:
test.returns.hist(bins=35);

In [ ]:
test['prediction'] = model.predict(test[cols])

In [ ]:
test['strategy'] = test['prediction'] * test['returns']

In [ ]:
accuracy_score(test['prediction'], test['direction'])

In [ ]:
test[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6));  # gross performance over time

<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>